# House Price Predictor

In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

In [30]:
# -----------------------------
# 1. Load the dataset
# -----------------------------
# Replace 'house_prices.csv' with the path to your dataset.
data = pd.read_csv('train.csv')
print(data)

       Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0       1          60       RL         65.0     8450   Pave   NaN      Reg   
1       2          20       RL         80.0     9600   Pave   NaN      Reg   
2       3          60       RL         68.0    11250   Pave   NaN      IR1   
3       4          70       RL         60.0     9550   Pave   NaN      IR1   
4       5          60       RL         84.0    14260   Pave   NaN      IR1   
..    ...         ...      ...          ...      ...    ...   ...      ...   
995   996          50       RL         51.0     4712   Pave   NaN      IR1   
996   997          20       RL          NaN    10659   Pave   NaN      IR1   
997   998          20       RL          NaN    11717   Pave   NaN      IR1   
998   999          30       RM         60.0     9786   Pave   NaN      Reg   
999  1000          20       RL         64.0     6762   Pave   NaN      Reg   

    LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeatu

In [31]:
# -----------------------------
# 2. Data Cleaning
# -----------------------------

data = pd.get_dummies(data, drop_first=True)

# Convert any boolean columns to integer (0 or 1)
data = data.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

# Select only numeric columns that have no missing data.
#numeric_cols = data.select_dtypes(include=[np.number]).columns
clean_numeric_cols = [col for col in data if data[col].isna().sum() == 0]
data_clean = data[clean_numeric_cols]


# Ensure that the target column 'price' is present.
if 'SalePrice' not in data_clean.columns:
    raise ValueError("The target column 'price' is not present in the complete numeric data.")


print(data_clean)


       Id  MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  \
0       1          60     8450            7            5       2003   
1       2          20     9600            6            8       1976   
2       3          60    11250            7            5       2001   
3       4          70     9550            7            5       1915   
4       5          60    14260            8            5       2000   
..    ...         ...      ...          ...          ...        ...   
995   996          50     4712            4            7       1946   
996   997          20    10659            5            6       1961   
997   998          20    11717            6            6       1970   
998   999          30     9786            3            4       1922   
999  1000          20     6762            7            5       2006   

     YearRemodAdd  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  ...  SaleType_ConLI  \
0            2003         706           0        150  ...             

In [32]:
# -----------------------------
# 3. Feature Selection
# -----------------------------
# Compute the correlation matrix using only the cleaned numeric data.
corr_matrix = data_clean.corr()

# Compute absolute correlations of features with the target and drop the target itself.
target_corr = corr_matrix['SalePrice'].drop('SalePrice').abs().sort_values(ascending=False)

# Print all features with their correlation values
print("All features sorted by correlation with SalePrice:")
print(target_corr)
target_corr.to_csv('target_corr.csv')

# Select only the top 4 features with the highest correlation with 'SalesPrice'
top15_features = target_corr.head(20).index
print("Selected top 15 features:", list(top15_features))

# Define input features (X) and target variable (y).
X = data_clean[top15_features].values
y = data_clean['SalePrice'].values.reshape(-1, 1)


All features sorted by correlation with SalePrice:
OverallQual        0.797666
GrLivArea          0.734997
GarageCars         0.658204
GarageArea         0.647953
TotalBsmtSF        0.642127
                     ...   
Condition1_RRNe    0.004722
Heating_GasW       0.000977
RoofMatl_Metal     0.000901
PoolQC_Fa          0.000506
BsmtFinSF2         0.000359
Name: SalePrice, Length: 228, dtype: float64
Selected top 15 features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'ExterQual_TA', 'TotRmsAbvGrd', 'FullBath', 'KitchenQual_TA', 'YearBuilt', 'YearRemodAdd', 'Foundation_PConc', 'Fireplaces', 'BsmtQual_TA', 'Neighborhood_NridgHt', 'ExterQual_Gd', 'BsmtFinType1_GLQ', 'GarageFinish_Unf', 'BsmtFinSF1']


In [33]:
# -----------------------------
# 4. Data Preprocessing
# -----------------------------
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features to improve training stability.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
# Convert the numpy arrays to PyTorch tensors.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create a TensorDataset and DataLoader for batch processing.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [35]:
# -----------------------------
# 5. Define the Neural Network Model
# -----------------------------
class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Output layer for regression
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HousePriceModel(input_dim=X_train.shape[1]).to(device)

In [36]:
# -----------------------------
# 6. Set Up Loss Function and Optimizer
# -----------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 7. Train the Model
# -----------------------------
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        

Epoch [10/1000], Loss: 34694294568.9600
Epoch [20/1000], Loss: 10684989460.4800
Epoch [30/1000], Loss: 5381788226.5600
Epoch [40/1000], Loss: 4426262231.0400
Epoch [50/1000], Loss: 3673734545.9200
Epoch [60/1000], Loss: 3120821808.6400
Epoch [70/1000], Loss: 2744703677.4400
Epoch [80/1000], Loss: 2485946850.5600
Epoch [90/1000], Loss: 2291427685.1200
Epoch [100/1000], Loss: 2137724248.3200
Epoch [110/1000], Loss: 2006037210.8800
Epoch [120/1000], Loss: 1893887235.2000
Epoch [130/1000], Loss: 1788568791.0400
Epoch [140/1000], Loss: 1699658166.4000
Epoch [150/1000], Loss: 1616850315.5200
Epoch [160/1000], Loss: 1543020458.2400
Epoch [170/1000], Loss: 1477358470.4000
Epoch [180/1000], Loss: 1413644547.8400
Epoch [190/1000], Loss: 1366041763.8400
Epoch [200/1000], Loss: 1319542188.8000
Epoch [210/1000], Loss: 1280084104.9600
Epoch [220/1000], Loss: 1247147504.6400
Epoch [230/1000], Loss: 1214899934.0800
Epoch [240/1000], Loss: 1186276071.6800
Epoch [250/1000], Loss: 1163142685.7600
Epoch [

In [37]:
# 8. Evaluate the Model
# -----------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor.to(device))
    test_loss = criterion(predictions, y_test_tensor.to(device)).item()
    print("Test Mean Squared Error:", test_loss)

    # Convert predictions to NumPy array and print
    predictions_np = predictions.cpu().numpy()
    print("Predictions (NumPy):", predictions_np)

# Optionally, to evaluate using scikit-learn's MSE:
mse = mean_squared_error(y_test, predictions_np)
print("Test MSE (scikit-learn):", mse)
#Test Mean Squared Error: 935741376.0

Test Mean Squared Error: 684468416.0
Predictions (NumPy): [[157946.58 ]
 [250775.48 ]
 [116040.23 ]
 [192008.66 ]
 [138855.38 ]
 [293779.1  ]
 [132045.83 ]
 [142381.1  ]
 [224118.78 ]
 [139621.19 ]
 [162906.   ]
 [104452.195]
 [ 62525.14 ]
 [190688.78 ]
 [255753.62 ]
 [134454.47 ]
 [214582.44 ]
 [121251.93 ]
 [114382.734]
 [207678.56 ]
 [229308.72 ]
 [230354.02 ]
 [127279.69 ]
 [431338.03 ]
 [ 93710.1  ]
 [117838.41 ]
 [165456.44 ]
 [134990.33 ]
 [188481.25 ]
 [308703.78 ]
 [131827.88 ]
 [198468.06 ]
 [259954.34 ]
 [114897.586]
 [185073.94 ]
 [140572.88 ]
 [158828.25 ]
 [124681.23 ]
 [147375.16 ]
 [120160.91 ]
 [176396.44 ]
 [225934.83 ]
 [123634.72 ]
 [169785.94 ]
 [144383.7  ]
 [102728.79 ]
 [193333.4  ]
 [121578.984]
 [129787.266]
 [118941.85 ]
 [151764.88 ]
 [106150.875]
 [318427.66 ]
 [161687.89 ]
 [134116.06 ]
 [ 92226.01 ]
 [114461.734]
 [125986.45 ]
 [213159.53 ]
 [185527.97 ]
 [119809.67 ]
 [127961.49 ]
 [223599.78 ]
 [149274.78 ]
 [230880.75 ]
 [300626.03 ]
 [180397.7  ]
 [25